In [1]:
using Pkg
Pkg.activate(".")


  Activating project at `~/workspace/uni/spatial_temporal_data/spatio_temporal_data-rainforest_development`
Precompiling project...
  ✓ libsixel_jll
  ✓ Gettext_jll
  ✓ Fontconfig_jll
  ✓ Glib_jll
  ? ImageBase
  ? JpegTurbo
  ✓ Cairo_jll
  ✓ HarfBuzz_jll
  ✓ Pango_jll
  ✓ MeshIO
  ✓ Cairo
  ? Sixel
  ✓ ImageAxes
  ✓ ImageMetadata
  ✓ Netpbm
  ✓ ImageIO
  ✓ Makie
  ✓ GeoMakie
  ✓ CairoMakie
  ✓ GLMakie
  17 dependencies successfully precompiled in 298 seconds. 253 already precompiled.
  3 dependencies failed but may be precompilable after restarting julia
  3 dependencies had warnings during precompilation:
┌ Sixel [45858cf5-a6b0-47a3-bbea-62219f50df47]
│  ┌ Warning: The call to compilecache failed to create a usable precompiled cache file for ImageCore [a09fc81d-aa75-5fe9-8630-4744c3626534]
│  │   exception = Required dependency ColorVectorSpace [c3611d14-8923-5661-9e6a-0046d554d3a4] failed to load from a cache file.
│  └ @ Base loading.jl:1818
└  
┌ JpegTurbo [b835a17e-a41a-41e7-81f0

In [2]:
include("src/rainforest_lib.jl")
using .Rainforestlib


[ Info: Precompiling Zarr [0a941bbe-ad1d-11e8-39d9-ab76183a1d99]
[ Info: Precompiling YAXArrays [c21b50f5-aa40-41ea-b809-c0f5e47bfa5c]
[ Info: Precompiling GeoMakie [db073c08-6b98-4ee5-b6a4-5efafb3259c6]
[ Info: Precompiling GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a]


In [5]:
using Zarr
using YAXArrays

lc = Rainforestlib.get_lcc_datacube()

filtered_dc = Rainforestlib.rough_spatial_filter(lc; time_bounds = (Date(2010), Date(2021)))

small_soy_region_dc = Rainforestlib.rough_spatial_filter(
    lc;
    lon_bounds = (-57, -54),
    lat_bounds = (-10, -6), 
    time_bounds = (Date(2010), Date(2021))
)

YAXArray with the following dimensions
lon                 Axis with 1080 Elements from -56.998611111111096 to -54.001388888888876
lat                 Axis with 1440 Elements from -6.001388888888883 to -9.998611111111105
time                Axis with 11 Elements from 2010-01-01T00:00:00 to 2020-01-01T00:00:00
name: lccs_class
Total size: 16.31 MB


In [3]:
include("src/LCCSClasses.jl")
using .LCCSClasses

# set a set of accepted values
rainforest_values = Set(LCCSClasses.categories["rainforest"].lccs_classes)

forest_values = Set(LCCSClasses.categories["forest"].lccs_classes)

all_forest_values = union(forest_values, rainforest_values)

Set{String} with 13 elements:
  "tree_needleleaved_evergreen_open"
  "tree_needleleaved_deciduous_closed"
  "tree_needleleaved_deciduous_open"
  "tree_needleleaved_deciduous_closed_to_open"
  "tree_broadleaved_deciduous_open"
  "tree_broadleaved_deciduous_closed"
  "tree_broadleaved_evergreen_closed_to_open"
  "tree_cover_flooded_saline_water"
  "tree_needleleaved_evergreen_closed"
  "tree_mixed"
  "tree_broadleaved_deciduous_closed_to_open"
  "tree_cover_flooded_fresh_or_brakish_water"
  "tree_needleleaved_evergreen_closed_to_open"

In [ ]:
# WARNING: do not try this with the huge datacube, it wont work. The in function needs to be ported to datacubes, the native one is not working
# to create the world map, simply set local_map=false
global_figure = Rainforestlib.build_figure_by_lcc_classes(lc, rainforest_values; local_map = false, set_nan = true)

In [ ]:
using .Rainforestlib

# local is the default, and makes the map as large as the datapoints
# but can me changed by setting 

using Colors, ColorSchemes

custom_colormap = [
    RGB(1.0, 1.0, 1.0),  # Red
    RGB(0.0, 0.0, 1.0),  # Blue
    RGB(0.0, 1.0, 0.0)   # Green
]

local_figure = Rainforestlib.build_figure_by_lcc_classes(small_soy_region_dc, all_forest_values; colormap = custom_colormap, colorrange = (0,1), set_nan = true)

In [ ]:
using .Rainforestlib

local_figure = Rainforestlib.build_figure_all_classes(filtered_dc; set_nan = true)

In [ ]:
Rainforestlib.build_plots_over_time(filtered_dc, all_forest_values; resolution=(2000, 2000), set_nan = true)

In [6]:
# build one figure with diffs for each timestep

using .Rainforestlib
using Colors, ColorSchemes
using GLMakie


custom_colormap = [
    RGB(1.0, 1.0, 1.0),  # White
    RGB(0.0, 1.0, 0.0),  # Green
    RGB(0.0, 0.0, 1.0),  # Blue
    RGB(1.0, 0.0, 0.0),  # Red
]

Rainforestlib.build_diff_figures_over_time(small_soy_region_dc, Rainforestlib.LCCSClasses.categories["rainforest"], "imgs"; name_base = "diff_first_year_small_soy_region", resolution = (3000, 3000))


Number of rainforest pixels in 2010-01-01T00:00:00: 1.325171e6


┌ Warning: Could not find font regular, using TeX Gyre Heros Makie
└ @ Makie ~/.julia/packages/Makie/iECbF/src/conversions.jl:993


Number of rainforest pixels in 2011-01-01T00:00:00: 1.324896e6
Number of rainforest pixels in 2012-01-01T00:00:00: 1.324261e6
Number of rainforest pixels in 2013-01-01T00:00:00: 1.32273e6
Number of rainforest pixels in 2014-01-01T00:00:00: 1.322726e6
Number of rainforest pixels in 2015-01-01T00:00:00: 1.322725e6
Number of rainforest pixels in 2016-01-01T00:00:00: 1.321695e6
Number of rainforest pixels in 2017-01-01T00:00:00: 1.3214e6
Number of rainforest pixels in 2018-01-01T00:00:00: 1.321067e6
Number of rainforest pixels in 2019-01-01T00:00:00: 1.320718e6
Number of rainforest pixels in 2020-01-01T00:00:00: 1.320511e6
